In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ignore harmless warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
root = '../data/cnn_files/CATS_DOGS/'
train_data = datasets.ImageFolder(os.path.join(root, 'train'), transform=train_transform)
test_data = datasets.ImageFolder(os.path.join(root, 'test'), transform=test_transform)

torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [ ]:
AlexNet_model = models.alexnet(pretrained=True)
AlexNet_model

In [ ]:
for param in AlexNet_model.parameters():
    param.requires_grad = False

In [ ]:
torch.manual_seed(42)

AlexNet_model.classifier = nn.Sequential(nn.Linear(in_features=9216, out_features=1024),
                                        nn.ReLU(),
                                        nn.Dropout(0.5),
                                        nn.Linear(in_features=1024, out_features=2),
                                        nn.LogSoftmax(dim=1))
AlexNet_model

In [ ]:
for param in AlexNet_model.parameters():
    print(param.numel())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(AlexNet_model.classifier.parameters(), lr=0.001)

In [ ]:
import time
start_time = time.time()

epoch = 1
train_losses_list = []
test_losses_list = []
train_corrects_list = []
test_corrects_list = []

for i in range(epoch):
    train_epoch_corrects = 0
    test_epoch_corrects = 0
    
    for batch_num, (X_train, y_train) in enumerate(train_loader):
        batch_num += 1
        
        y_pred = AlexNet_model.forward(X_train)
        loss = criterion(y_pred, y_train)
        
        predicted = torch.max(y_pred.data, dim=1)[1]
        batch_corrects = (predicted == y_train).sum()
        train_epoch_corrects += batch_corrects
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_num % 100 == 0:
            acc = (train_epoch_corrects / (batch_num*10))*100
            print(f'epoch:{i}, batch:{batch_num}, loss:{loss}, acc:{acc}')
        
    train_losses_list.append(loss)
    train_corrects_list.append(train_epoch_corrects)
    
    with torch.no_grad():
        for batch_num, (X_test, y_test) in enumerate(test_loader):
            batch_num += 1
            
            y_eval = AlexNet_model.forward(X_test)

            predicted = torch.max(y_eval.data, dim=1)[1]
            batch_corrects = (predicted == y_test).sum()
            test_epoch_corrects += batch_corrects
            
        loss = criterion(y_eval, y_test)
        test_losses_list.append(loss)
        test_corrects_list.append(test_epoch_corrects)

duration = time.time() - start_time
print(f'Total time is {duration/60} mins')

In [ ]:
torch.save(AlexNet_model.state_dict(), '../models/my_cat_dog_Alexnet_1.pt')